#**Question 1**

Cholesky decomposition

In [1]:
import numpy as np

def Cholesky(a):
    a = np.array(a, float)
    L = np.zeros_like(a)
    n = np.shape(a)[0]

    for j in range(n):
        for i in range(j, n):
            if i == j:
                sumk = 0.0
                for k in range(j):
                    sumk = sumk + L[i][k] ** 2
                L[i][j] = np.sqrt(a[i][j] - sumk)
            else:
                sumk = 0.0
                for k in range(j):
                    sumk = sumk + L[i][k] * L[j][k]
                L[i][j] = (a[i][j] - sumk) / L[j][j]

    L_t = L.T

    def forsub(L, bs):
        n = bs.size
        xs = np.zeros(n)
        for i in range(n):
            xs[i] = (bs[i] - L[i, :i] @ xs[:i]) / L[i, i]
        return xs

    def backsub(U, bs):
        n = bs.size
        xs = np.zeros(n)
        for i in reversed(range(n)):
            xs[i] = (bs[i] - U[i, i + 1:] @ xs[i + 1:]) / U[i, i]
        return xs
    y=forsub(L,b)
    z=backsub(L_t,y)
    return z

A = np.array([[4, -1, 0, -1, 0, 0], [-1, 4, -1, 0, -1, 0], [0, -1, 4, 0, 0, -1],
              [-1, 0, 0, 4, -1, 0], [0, -1, 0, -1, 4, -1], [0, 0, -1, 0, -1, 4]])
b=np.array([2,1,2,2,1,2])
print(Cholesky(A))


[1. 1. 1. 1. 1. 1.]


In [2]:
def gauss_sidel(A,b,x):
    n=0
    tol=100
    while tol>10**(-15):
        x0=x.copy()
        for i in range(len(x)):
            result=0
            for j in range(len(x)):
                result+=A[i,j]*x[j]
            x[i]=(b[i]-result+A[i,i]*x[i])/(A[i,i])
        tol=np.sum((x-x0)**2)
        n+=1
    return(x)
x=np.zeros(6)
print(gauss_sidel(A,b,x))

[1. 1. 1. 1. 1. 1.]


In [3]:
solution = np.linalg.solve(A, b)
print(solution)

[1. 1. 1. 1. 1. 1.]


#**Question 2**

In [4]:
import numpy as np

A=np.array([[2,5,1,3,13],[4,0,4,10,1],[0.0,4.0,2.0,0,1],[11,3,0,1,2],[3,2,7,1,0]],dtype=float)
b=np.array([20,15,92,51,15],dtype=float)

def ludec(A):
    n = A.shape[0]
    U = np.copy(A)
    L = np.identity(n)
    for j in range(n-1):
        for i in range(j+1,n):
            coeff = U[i,j]/U[j,j]
            U[i,j:] -= coeff*U[j,j:]
            L[i,j] = coeff
    return L, U
def forsub(L,bs):
    n = bs.size
    xs = np.zeros(n)
    for i in range(n):
        xs[i] = (bs[i] - L[i,:i]@xs[:i])/(L[i,i])
    return xs
def backsub(U,bs):
    n = bs.size
    xs = np.zeros(n)
    for i in reversed(range(n)):
        xs[i] = (bs[i] - U[i,i+1:]@xs[i+1:])/(U[i,i])
    return xs
def lusolve(A,bs):
    L, U = ludec(A)
    ys = forsub(L,bs)
    xs = backsub(U,ys)
    return xs

print(lusolve(A,b))

[-1.83590378 28.50565971 -6.02062257  5.64074991 -9.9813937 ]


In [5]:
import numpy as np
def gaussjordan(a,b):
    a=np.array(a,float)
    b=np.array(b,float)
    n=len(b)
    for k in range(n):
        if abs(a[k,k])<1.0e-12:
            for i in range(k+1,n):#row
                if abs(a[i,k])>abs(a[k,k]):
                    for j in range(k,n):#colummn
                        a[k,j],a[i,j]=a[i,j],a[k,j]
                    b[k],b[i]=b[i],b[k]
                    break
        pivot=a[k,k]
        for j in range(k,n):
            a[k,j]=a[k,j]/pivot
        b[k]=b[k]/pivot
        for i in range(n):
            if i==k or a[i,k]==0:continue
            factor=a[i,k]
            for j in range(k,n):
                a[i,j]=a[i,j]-factor*a[k,j]
            b[i]=b[i]-factor*b[k]
    return b
x=gaussjordan(A,b)
print(x)

[-1.83590378 28.50565971 -6.02062257  5.64074991 -9.9813937 ]


In [6]:
solution = np.linalg.solve(A, b)
print(solution)

[-1.83590378 28.50565971 -6.02062257  5.64074991 -9.9813937 ]


#**Question 3**

In [7]:
import numpy as np
def conjugate_gradient(A,b,x,tol=0.000001,itermax=54):
  r=b-A@x
  d=r
  i=0
  residue=np.sqrt(r@r)
  while i<=itermax and residue >= tol:
    alpha=(r@r)/(d@(A@d))
    x=x+alpha*d
    r1=r-alpha*(A@d)
    beta=(r1@r1)/(r@r)
    d=r1+beta*d
    r=r1
    residue=np.sqrt(r@r)
    i+=1
  return x
def conjugate_gradient_inverse(A):
  l=[]
  I=np.identity(A.shape[1])
  x0=0.5*np.ones(A.shape[1])
  for j in range(A.shape[1]):
    x=conjugate_gradient(A,I[:,j],x0)
    l.append(x)
  result=np.vstack(l)
  return result.T
def conjgrad(A, b, x):
    r = b - np.dot(A, x)
    p = r
    rsold = np.dot(np.transpose(r), r)

    for i in range(len(b)):
        Ap = np.dot(A, p)
        alpha = rsold / np.dot(np.transpose(p), Ap)
        x = x + np.dot(alpha, p)
        r = r - np.dot(alpha, Ap)
        rsnew = np.dot(np.transpose(r), r)
        if np.sqrt(rsnew) < 1e-9:
            break
        p = r + (rsnew/rsold)*p
        rsold = rsnew
    return x
import numpy as np
M=np.array([[2,-3,0,0,0,0],[-1,4,-1,0,-1,0],[0,-1,4,0,0,-1],[0,0,0,2,-3,0],[0,-1,0,-1,4,-1],[0,0,-1,0,-1,4]])
b=np.array([-(5/3),(2/3),3,(-4/3),(-1/3),(5/3)])
x0=np.ones(6)






In [8]:
conjgrad(M,b,x0)

array([-0.5597598 ,  0.15584588,  0.76952535, -1.28134912, -0.16611059,
        0.58875806])

In [9]:
print(conjugate_gradient(M,b,x0))

[-0.3447479   0.32688228  0.99761917 -0.68164938 -0.00712292  0.66452815]


In [10]:
solution = np.linalg.solve(M, b)
print(solution)

[-3.33333333e-01  3.33333333e-01  1.00000000e+00 -6.66666667e-01
  1.60032148e-17  6.66666667e-01]


In [11]:
Q=conjugate_gradient_inverse(M)
print(Q)


[[0.91297646 0.93578768 0.25615252 0.21230963 0.50834423 0.16956203]
 [0.28746764 0.59692854 0.17256342 0.13813613 0.3030059  0.11216624]
 [0.08350403 0.15499575 0.31979197 0.04920808 0.10121089 0.10776105]
 [0.21230963 0.50834423 0.16956203 0.91297646 0.93578768 0.25615252]
 [0.13813613 0.3030059  0.11216624 0.28746764 0.59692854 0.17256342]
 [0.04920808 0.10121089 0.10776105 0.08350403 0.15499575 0.31979197]]


In [12]:
S=np.linalg.inv(M)
print(S)

[[0.93506494 0.87012987 0.25974026 0.20779221 0.41558442 0.16883117]
 [0.29004329 0.58008658 0.17316017 0.13852814 0.27705628 0.11255411]
 [0.08658009 0.17316017 0.32034632 0.05627706 0.11255411 0.10822511]
 [0.20779221 0.41558442 0.16883117 0.93506494 0.87012987 0.25974026]
 [0.13852814 0.27705628 0.11255411 0.29004329 0.58008658 0.17316017]
 [0.05627706 0.11255411 0.10822511 0.08658009 0.17316017 0.32034632]]


#**Question 4**

In [13]:
import numpy as np
m=0.2
d=m**2-1
dia=d*np.eye(50,k=0,dtype=float)
A1=np.zeros((51,51),dtype=float)
L=0.5*np.ones(50)
for i in range(0,50):
  for j in range(0,50):
    if j==2*i:
      A1[i][j]=1.0/2
    else:
      A1[i][j]=0.0
A1= np.delete(A1, 0, axis=0)
A1 = np.delete(A1, 0, axis=1)
A2=np.zeros((50,50),dtype=float)
A2[0,:]=L
print(np.shape(A1))
A=A1+dia+A2
print(A)


(50, 50)
[[-0.46  1.    0.5  ...  0.5   0.5   0.5 ]
 [ 0.   -0.96  0.   ...  0.    0.    0.  ]
 [ 0.    0.   -0.96 ...  0.    0.    0.  ]
 ...
 [ 0.    0.    0.   ... -0.96  0.    0.  ]
 [ 0.    0.    0.   ...  0.   -0.96  0.  ]
 [ 0.    0.    0.   ...  0.    0.   -0.96]]


In [14]:
print(conjugate_gradient_inverse(A))

[[ 5.20948813e+08 -2.46095287e+08 -5.11582014e+06 ... -6.18875143e+16
  -1.34180393e+02 -1.61650836e+04]
 [ 3.11911914e+08 -3.27666887e+08 -2.84217450e+06 ... -2.93025556e+16
  -9.05966870e+01 -9.01421452e+03]
 [ 3.11911914e+08 -1.28036785e+08 -7.01891562e+06 ... -2.93025558e+16
  -9.05966870e+01 -9.01421452e+03]
 ...
 [ 2.33102578e+08 -9.23815189e+07 -2.00483543e+06 ... -6.35603837e+16
  -6.47196421e+01 -6.35788901e+03]
 [ 2.33102578e+08 -9.23815189e+07 -2.00483543e+06 ... -2.06141785e+16
  -1.99555633e+02 -6.35788901e+03]
 [ 2.33102578e+08 -9.23815189e+07 -2.00483543e+06 ... -2.06141785e+16
  -6.47196421e+01 -1.96033504e+04]]
